<a target="_blank" href="https://colab.research.google.com/github/gox6/colab-demos/blob/main/wip/polars-a-modern-dataframe-library.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<html>
<body>
    <div style="display: flex;">
        <a href="https://towardsdatascience.com/evaluate-rags-rigorously-or-perish-54f790557357" style="width: 45em; height: 15em; background-color: white; padding-left: 1em; display: flex; flex-direction: column; justify-content: center; color: inherit; text-decoration: none;">
            <p style="margin: 0; font-size: 1.5em; color: darkblue; font-weight: bold;">The notebook contains the code for Medium article</p>
            <p style="margin: 0; font-weight: bold; font-size: 1.5em; color: #3D4849;">Invent the title</p>
        </a>
        <a href="https://towardsdatascience.com/evaluate-rags-rigorously-or-perish-54f790557357" style="width: 30em; height: 15em;">
            <img src="https://images.unsplash.com/photo-1522991150037-09a660b94ba4?q=80&w=1470&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" style="width: 100%; height: 100%; object-fit: cover;">
        </a>
    </div>
</body>
</html>

# Project Setup

In [3]:
# Installing Python packages quietly

!pip install --quiet \
  httpx \
  pandas \
  polars \
  pyarrow \
  1> /dev/null

In [34]:
# Importing the packages
from getpass import getpass
from io import BytesIO
import os
from zipfile import ZipFile


import httpx
import pandas as pd
import polars as pl

In [5]:
# Managing secrets
# - If using Colab please use Colab Secrets
# - If running outside Colab please provide secrets as environmental variables
COLAB = os.getenv("COLAB_RELEASE_TAG") is not None

if COLAB:
    from google.colab import userdata, data_table
    runtime_info = "Colab runtime"

    # Enabling Colab's data formatter for pandas
    data_table.enable_dataframe_formatter()
else:
    runtime_info = "Non Colab runtime"

print(runtime_info)

Colab runtime


In [27]:
# Download a zip file into the buffer
url = "https://archive.ics.uci.edu/static/public/791/metropt+3+dataset.zip"
with httpx.Client() as client:
    response = client.get(url)
    buffer = BytesIO(response.content)

# Unzip into the current directory
with ZipFile(buffer) as zip_ref:
    # Extract all files to the current directory
    zip_ref.extractall('.')


In [38]:
%%timeit -r 2-
df_pl = pl.read_csv('MetroPT3(AirCompressor).csv')


1.23 s ± 73.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
%%timeit -r 20
df_pd = pd.read_csv('MetroPT3(AirCompressor).csv')

4.56 s ± 695 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [61]:
%%timeit -r 20
df_pl.describe()

893 ms ± 232 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


In [62]:
%%timeit  -r 20
df_pd.describe()

931 ms ± 118 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


In [ ]:
%%time
df_pd.mean()

In [ ]:
# Seeing news data
# if COLAB:
#   display(data_table.DataTable(news_pd, include_index=False, num_rows_per_page=5))
# else:
#   display(news_pd.head(5))

<html>
<body>
    <div style="display: flex;">
            <a href="https://medium.com/@jgrygolec" style="width: 45em; height: 15em; background-color: white; padding-left: 1em; display: flex; flex-direction: column; justify-content: center; color: inherit; text-decoration: none;">
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">Thank you for attention!</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849; font-weight: bold;">Click to see more of my articles on Medium.</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">BR,</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">Jarek Grygolec</p>
        </a>
        <a href="https://medium.com/@jgrygolec" style="width: 30em; height: 15em; background-repeat: no-repeat; background-size: cover; background-position: center;">
                    <img src="https://miro.medium.com/v2/resize:fit:3992/0*WCQwsoZC0FA2-haq" style="width: 100%; height: 100%; object-fit: cover;"></a>
    </div>
</body>
</html>